In [ ]:
from Raspi_MotorHAT import Raspi_MotorHAT
import keyboard
import threading
import time
import cv2
import RPi.GPIO as GPIO
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
mh = Raspi_MotorHAT(addr=0x6f)


def motor_go():   # 직진하는 함수
    motor1 = mh.getMotor(1)              # 각 바퀴에 맞는 포트번호 확인
    motor1.setSpeed(50)                  # 각 바퀴의 속도 제어
    motor1.run(Raspi_MotorHAT.FORWARD)   # Raspi_MotorHAT 제어
    motor2 = mh.getMotor(2)
    motor2.setSpeed(50)
    motor2.run(Raspi_MotorHAT.FORWARD)
    motor3 = mh.getMotor(3)
    motor3.setSpeed(50)
    motor3.run(Raspi_MotorHAT.FORWARD)
    motor4 = mh.getMotor(4)
    motor4.setSpeed(50)
    motor4.run(Raspi_MotorHAT.FORWARD)

def motor_stop():  # 정지하는 함수
    motor1 = mh.getMotor(1)
    motor1.setSpeed(50)
    motor1.run(Raspi_MotorHAT.RELEASE)
    motor2 = mh.getMotor(2)
    motor2.setSpeed(50)
    motor2.run(Raspi_MotorHAT.RELEASE)
    motor3 = mh.getMotor(3)
    motor3.setSpeed(50)
    motor3.run(Raspi_MotorHAT.RELEASE)
    motor4 = mh.getMotor(4)
    motor4.setSpeed(50)
    motor4.run(Raspi_MotorHAT.RELEASE)
    
def motor_right():  # 우회전 함수
    motor1 = mh.getMotor(1)
    motor1.setSpeed(70)
    motor1.run(Raspi_MotorHAT.FORWARD)
    motor2 = mh.getMotor(2)
    motor2.setSpeed(15)
    motor2.run(Raspi_MotorHAT.FORWARD)
    motor3 = mh.getMotor(3)
    motor3.setSpeed(70)
    motor3.run(Raspi_MotorHAT.FORWARD)
    motor4 = mh.getMotor(4)
    motor4.setSpeed(15)
    motor4.run(Raspi_MotorHAT.FORWARD)  
    
def motor_left():   # 좌회전 함수
    motor1 = mh.getMotor(1)
    motor1.setSpeed(15)
    motor1.run(Raspi_MotorHAT.FORWARD)
    motor2 = mh.getMotor(2)
    motor2.setSpeed(70)
    motor2.run(Raspi_MotorHAT.FORWARD)
    motor3 = mh.getMotor(3)
    motor3.setSpeed(15)
    motor3.run(Raspi_MotorHAT.FORWARD)
    motor4 = mh.getMotor(4)
    motor4.setSpeed(70)
    motor4.run(Raspi_MotorHAT.FORWARD)

# 원하는 이미지와 인덱스 딕셔너리 설정 - id와 name을 매칭, 약 90개의 객체를 매칭
classNames = {0: 'background',
              1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus',
              7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant',
              13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat',
              18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear',
              24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 31: 'handbag',
              32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard',
              37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove',
              41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle',
              46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon',
              51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange',
              56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut',
              61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed',
              67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse',
              75: 'remote', 76: 'keyboard', 77: 'cell phone', 78: 'microwave', 79: 'oven',
              80: 'toaster', 81: 'sink', 82: 'refrigerator', 84: 'book', 85: 'clock',
              86: 'vase', 87: 'scissors', 88: 'teddy bear', 89: 'hair drier', 90: 'toothbrush'}

### OPENCV_DNN_CODE
def id_class_name(class_id, classes):   # key, value에 맞는 item 매칭해주는 함수, 즉 id를 이름으로 변경
    for key, value in classes.items():
        if class_id == key:
            return value

def opencvdnn_thread():                 # opencv DNN 모델 함수
    global image
    global image_ok
    global box_size
    global carState
    model = cv2.dnn.readNetFromTensorflow('/home/jh/model/frozen_inference_graph.pb',          # 가중치 파일과 샘플 코드 불러온다
                                      '/home/jh/model/ssd_mobilenet_v2_coco_2018_03_29.pbtxt')
 
    while True:
        if image_ok == 1:
            imagednn = image
            image_height, image_width, _ = imagednn.shape                                      # imagednn에서 높이와 넓이 를 가져온다.
            
            model.setInput(cv2.dnn.blobFromImage(imagednn, size=(250, 250), swapRB=True))      # imagednn 전처리, 사이즈가 크면 시간이 오래 걸리기 때문
            output = model.forward()                                                           # 전처리된 결과
            # print(output[0,0,:,:].shape)
            for detection in output[0, 0, :, :]:
                confidence = detection[2]
                if confidence > .5:                                                            # 신뢰도가 50% 이상이면 조건이 참이 된다.
                    class_id = detection[1]
                    class_name=id_class_name(class_id,classNames)
                    if class_name == "person" or class_name == "dog":         # 사람 또는 개를 발견시 정지
                        print(str(str(class_id) + " " + str(detection[2])  + " " + class_name))
                        box_x = detection[3] * image_width
                        box_y = detection[4] * image_height
                        box_width = detection[5] * image_width
                        box_height = detection[6] * image_height
                        box_size = box_width * box_height
                        #print("box_size:",box_size)
                        
                        carState = "stop"                        # carState를 정지로 바꿔준다.
                        print("auto stop")
                
                        cv2.rectangle(imagednn, (int(box_x), int(box_y)), (int(box_width), int(box_height)), (23, 230, 210), thickness=1) # 인식시 사각형 범위 지정
                        cv2.putText(imagednn,class_name ,(int(box_x), int(box_y+.05*image_height)),cv2.FONT_HERSHEY_SIMPLEX,(.005*image_width),(0, 0, 255)) # object 표시
                        
                    if class_name == "bird":                                # 새일때는 계속 주행한다.                  
                        print(str(str(class_id) + " " + str(detection[2])  + " " + class_name))
                        box_x = detection[3] * image_width
                        box_y = detection[4] * image_height
                        box_width = detection[5] * image_width
                        box_height = detection[6] * image_height
                        box_size = box_width * box_height
                        #print("box_size:",box_size)
                        
                        carState = "go"                        # carState를 직진으로 바꿔준다.
                        print("auto go")
                
                        cv2.rectangle(imagednn, (int(box_x), int(box_y)), (int(box_width), int(box_height)), (23, 230, 210), thickness=1) # 인식시 사각형 범위 지정
                        cv2.putText(imagednn,class_name ,(int(box_x), int(box_y+.05*image_height)),cv2.FONT_HERSHEY_SIMPLEX,(.005*image_width),(0, 0, 255)) # object 표시
            
            #cv2.imshow('imagednn', imagednn)
            image_ok = 0
        


In [ ]:
### TRAIN_CODE
speed = True
def main():
    camera = cv2.VideoCapture(0)
    camera.set(3,640)
    camera.set(4,480)
    filepath = '/home/jh/ai_car/train'          # 이 경로로 이미지가 저장된다. ex) train_045
    i = 0                                       # 이미지 중복 저장을 막기위해 i에 변수값을 지정
    carState = 'stop'
    
    while (camera.isOpened()):                  # 카메라가 켜지는 동안 샐행한다.
        keyValue = cv2.waitKey(10)
        #print(str(keyValue))
        
        if keyboard.is_pressed("p"):
            break
        elif keyboard.is_pressed("w"):          # w입력시 직진, carstate가 go로 변경
            print('go')
            carState = 'go'
            motor_go()
        elif keyboard.is_pressed("a"):          # a입력시 좌회전, carstate가 left로 변경
            print('left')
            carState = 'left'
            motor_left()
        elif keyboard.is_pressed("d"):          # d입력시 우회전, carstate가 right로 변경
            print('right')
            carState = 'right'
            motor_right()
        elif keyboard.is_pressed("s"):          # s입력시 정지, carstate가 stop로 변경
            print('stop')
            carState = 'stop'
            motor_stop()
            
            
        _,image = camera.read()
        #cv2.imshow('Original',image)
        
        height, _, _=image.shape
        save_image = image[int(height/2):,:,:]
        save_image = cv2.cvtColor(save_image, cv2.COLOR_BGR2YUV)   # 트랙을 보다 선명하게
        save_image = cv2.resize(save_image, (200,66))              # 빠른 학습을 위해, 보다 차선을 확실히 확인하기 위해 사이즈 조정
        save_image = cv2.GaussianBlur(save_image, (5,5), 0)        # 필터를 사용해 이미지 인식 성능 개선
        _,save_image = cv2.threshold(save_image, 160, 255, cv2.THRESH_BINARY_INV)  # 임계점을 조정해 트랙을 선명하게 볼수 있도록
        cv2.imshow('Save', save_image)
        
        if carState == 'left':
            cv2.imwrite('%s_%05d_%03d.png' %(filepath,i,45),save_image)    # 이미지가 저장된다 ex) train_00001_045.png
            i += 1                                                         # i값이 증가함에 따라 중복 저장 예방
        elif carState == 'right':
            cv2.imwrite('%s_%05d_%03d.png' %(filepath,i,135),save_image)
            i += 1
        elif carState == 'go':
            cv2.imwrite('%s_%05d_%03d.png' %(filepath,i,90),save_image)
            i +=1
        
    cv2.destroyAllWindows()
    
if __name__ == '__main__':
    main()

In [ ]:
### TEST_CODE
def img_preprocess(image):                         # 이미지를 학습된 데이터 형식에 맞게 보정하는 함수
    height, _, _ = image.shape
    image = image[int(height/2):,:,:]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)  # 트랙을 보다 선명하게
    image = cv2.resize(image, (200,66))
    image = cv2.GaussianBlur(image,(5,5),0)         # 트랙을 선명하게 필터 추가
    _,image = cv2.threshold(image,160,255,cv2.THRESH_BINARY_INV)  # 화질의 개선처리, 임계점에 따라 화질이 달라짐, 숫자를 조정하며 최적의 임계점을 찾는다.
    image = image / 255                             # 이미지 정규화
    return image                                    

camera = cv2.VideoCapture(-1)
camera.set(3, 640)
camera.set(4, 480)

        
def main():
    
    model_path = '/home/jh/model/1205-2.h5'         # 학습 모델이 위치한 경로를 지정한다.
    model = load_model(model_path)                  # 모델을 불러온다.
    
    carState = "stop"
    
    try:
        while True:
            keyValue = cv2.waitKey(1)
        
            if keyboard.is_pressed("p"):
                break
            elif keyboard.is_pressed("w"):
                print("go")
                carState = "go"
            elif keyboard.is_pressed("s"):
                print("stop")
                carState = "stop"
                
            _, image = camera.read()
            preprocessed = img_preprocess(image)           # 보정한 이미지를 저장
            cv2.imshow('pre', preprocessed)                # 보정한 이미지를 opencv창으로 출력
            
            X = np.asarray([preprocessed])                 # X값에 형식에 맞춰 데이터를 넣는다.
            steering_angle = int(model.predict(X)[0])      # 각도를 예측해준다(조향각도)
            print("predict angle:",steering_angle)         # 예측한 각도를 출력
                
            if carState == "go":
                if steering_angle >= 80 and steering_angle <= 105: # 조향각도가 80이상 105이하라면 계속 직진
                    print("go")
                    motor_go()
                elif steering_angle > 105:                         # 조향각도가 105 이상일때 우회전
                    print("right")
                    motor_right()
                elif steering_angle < 80:                          # 조향각도가 80 이상일때 좌회전
                    print("left")
                    motor_left()
            elif carState == "stop":
                motor_stop()
            
    except KeyboardInterrupt:
        pass

if __name__ == '__main__':
    main()
    cv2.destroyAllWindows()